# Read D3D uvp, files

In [1]:
import pandas as pd

import numpy as np

import xarray as xr

In [2]:
df = pd.read_csv('../../examples/test/d3d/u.amu',header=0, names=['data'], index_col=None, low_memory=False)

In [3]:
df.head()

,data
0,Filetype = meteo_on_equidistant_grid
1,n_cols = 39
2,n_rows = 33
3,grid_unit = degree
4,x_llcenter = 160


In [4]:
n0 = df[df.data.str.contains('TIME')].index

In [5]:
n0

Int64Index([ 12,  46,  80, 114, 148, 182, 216, 250, 284, 318, 352, 386, 420,
            454, 488, 522, 556],
           dtype='int64')

In [6]:
d1 = df.loc[0:n0[0]-1,'data'].str.split('=', 2, expand=True)
d1

,0,1
0,Filetype,meteo_on_equidistant_grid
1,n_cols,39
2,n_rows,33
3,grid_unit,degree
4,x_llcenter,160
5,dx,0.5
6,y_llcenter,-49
7,dy,0.5
8,NODATA_value,-9999.000
9,n_quantity,1


In [7]:
d1.columns=['key','value']
d1

,key,value
0,Filetype,meteo_on_equidistant_grid
1,n_cols,39
2,n_rows,33
3,grid_unit,degree
4,x_llcenter,160
5,dx,0.5
6,y_llcenter,-49
7,dy,0.5
8,NODATA_value,-9999.000
9,n_quantity,1


In [8]:
d1.key = d1.key.str.strip()
d1

,key,value
0,Filetype,meteo_on_equidistant_grid
1,n_cols,39
2,n_rows,33
3,grid_unit,degree
4,x_llcenter,160
5,dx,0.5
6,y_llcenter,-49
7,dy,0.5
8,NODATA_value,-9999.000
9,n_quantity,1


In [9]:
d1.value = d1.value.str.strip()
d1

,key,value
0,Filetype,meteo_on_equidistant_grid
1,n_cols,39
2,n_rows,33
3,grid_unit,degree
4,x_llcenter,160
5,dx,0.5
6,y_llcenter,-49
7,dy,0.5
8,NODATA_value,-9999.000
9,n_quantity,1


In [10]:
attrs = dict(zip(d1.key, d1.value))
attrs

{'Filetype': 'meteo_on_equidistant_grid',
 'n_cols': '39',
 'n_rows': '33',
 'grid_unit': 'degree',
 'x_llcenter': '160',
 'dx': '0.5',
 'y_llcenter': '-49',
 'dy': '0.5',
 'NODATA_value': '-9999.000',
 'n_quantity': '1',
 'quantity1': 'x_wind',
 'unit1': 'm s-1'}

In [11]:
for key in ['n_cols','n_rows','n_quantity']:
    attrs[key] = int(attrs[key])

In [12]:
for key in ['x_llcenter','dx','y_llcenter','dy','NODATA_value']:
    attrs[key] = float(attrs[key])

In [13]:
attrs

{'Filetype': 'meteo_on_equidistant_grid',
 'n_cols': 39,
 'n_rows': 33,
 'grid_unit': 'degree',
 'x_llcenter': 160.0,
 'dx': 0.5,
 'y_llcenter': -49.0,
 'dy': 0.5,
 'NODATA_value': -9999.0,
 'n_quantity': 1,
 'quantity1': 'x_wind',
 'unit1': 'm s-1'}

In [14]:
d2 = df.loc[n0, 'data'].str.split('=', 2, expand=True)
d2 = d2.drop(d2.columns[0], axis=1)
d2.columns=['data']
d2 = d2.loc[:,'data'].str.split(' ', 4, expand=True)
d2 = d2.drop(d2.columns[[0,2,3]], axis=1)
d2.columns = ['hours','time0']
d2.hours = d2.hours.apply(pd.to_numeric)
d2.time0 = pd.to_datetime(d2.time0.values)
d2 = d2.reset_index(drop=True)
d2

,hours,time0
0,163632,2000-01-01 00:00:00+00:00
1,163635,2000-01-01 00:00:00+00:00
2,163638,2000-01-01 00:00:00+00:00
3,163641,2000-01-01 00:00:00+00:00
4,163644,2000-01-01 00:00:00+00:00
5,163647,2000-01-01 00:00:00+00:00
6,163650,2000-01-01 00:00:00+00:00
7,163653,2000-01-01 00:00:00+00:00
8,163656,2000-01-01 00:00:00+00:00
9,163659,2000-01-01 00:00:00+00:00


In [15]:
d2.time0[0]

Timestamp('2000-01-01 00:00:00+0000', tz='UTC')

In [16]:
time=[]
for i in range(d2.shape[0]):
    time.append(d2.time0[0] + pd.DateOffset(hours=int(d2.loc[i,'hours'])))
time

[Timestamp('2018-09-01 00:00:00+0000', tz='UTC'),
 Timestamp('2018-09-01 03:00:00+0000', tz='UTC'),
 Timestamp('2018-09-01 06:00:00+0000', tz='UTC'),
 Timestamp('2018-09-01 09:00:00+0000', tz='UTC'),
 Timestamp('2018-09-01 12:00:00+0000', tz='UTC'),
 Timestamp('2018-09-01 15:00:00+0000', tz='UTC'),
 Timestamp('2018-09-01 18:00:00+0000', tz='UTC'),
 Timestamp('2018-09-01 21:00:00+0000', tz='UTC'),
 Timestamp('2018-09-02 00:00:00+0000', tz='UTC'),
 Timestamp('2018-09-02 03:00:00+0000', tz='UTC'),
 Timestamp('2018-09-02 06:00:00+0000', tz='UTC'),
 Timestamp('2018-09-02 09:00:00+0000', tz='UTC'),
 Timestamp('2018-09-02 12:00:00+0000', tz='UTC'),
 Timestamp('2018-09-02 15:00:00+0000', tz='UTC'),
 Timestamp('2018-09-02 18:00:00+0000', tz='UTC'),
 Timestamp('2018-09-02 21:00:00+0000', tz='UTC'),
 Timestamp('2018-09-03 00:00:00+0000', tz='UTC')]

In [17]:
d2['time']=time
d2

,hours,time0,time
0,163632,2000-01-01 00:00:00+00:00,2018-09-01 00:00:00+00:00
1,163635,2000-01-01 00:00:00+00:00,2018-09-01 03:00:00+00:00
2,163638,2000-01-01 00:00:00+00:00,2018-09-01 06:00:00+00:00
3,163641,2000-01-01 00:00:00+00:00,2018-09-01 09:00:00+00:00
4,163644,2000-01-01 00:00:00+00:00,2018-09-01 12:00:00+00:00
5,163647,2000-01-01 00:00:00+00:00,2018-09-01 15:00:00+00:00
6,163650,2000-01-01 00:00:00+00:00,2018-09-01 18:00:00+00:00
7,163653,2000-01-01 00:00:00+00:00,2018-09-01 21:00:00+00:00
8,163656,2000-01-01 00:00:00+00:00,2018-09-02 00:00:00+00:00
9,163659,2000-01-01 00:00:00+00:00,2018-09-02 03:00:00+00:00


In [18]:
d3 = df.drop(np.arange(0,n0[0]))
d3 = d3.drop(n0)
d3

,data
13,12.573 13.373 13.673 13.673 13.573 13.073 12.5...
14,11.873 12.573 13.773 13.873 13.673 13.073 12.5...
15,11.273 12.173 12.773 13.273 13.073 12.773 12.4...
16,11.573 11.873 11.973 11.973 12.273 12.073 12.1...
17,13.573 13.173 12.273 11.473 11.573 11.573 11.9...
18,15.073 14.773 13.473 11.973 11.373 11.373 11.5...
19,15.873 15.273 14.373 12.273 11.273 11.173 10.9...
20,16.173 15.573 14.573 12.273 11.273 10.873 10.3...
21,15.873 15.273 14.173 12.073 11.273 10.773 9.57...
22,14.873 14.173 13.273 11.673 11.273 10.873 9.87...


In [19]:
d3.values.shape

(561, 1)

In [52]:
%time
data = []
for i in range(d3.values.shape[0]):
    row = d3.values[i][0].split(' ')
    row = [np.float(x) for x in row]
    data.append(row)


data = np.array(data)
data.shape

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


(561, 39)

In [55]:
#or with pandas one-liner
%time
data = d3[d3.columns[0]].str.split(' ', attrs['n_cols'], expand=True).to_numpy().astype(float)
data.shape

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


(561, 39)

In [56]:
data = data.reshape(d2.shape[0],attrs['n_rows'], attrs['n_cols'])

In [57]:
data

array([[[12.573, 13.373, 13.673, ..., 12.073, 12.373, 12.673],
        [11.873, 12.573, 13.773, ..., 10.973, 11.473, 11.873],
        [11.273, 12.173, 12.773, ...,  9.873, 10.273, 10.473],
        ...,
        [ 4.173,  3.973,  3.673, ...,  0.373, -1.327, -4.127],
        [ 4.273,  4.073,  3.773, ..., -4.627, -5.427, -5.727],
        [ 4.473,  4.273,  3.873, ..., -5.127, -5.627, -5.327]],

       [[13.346, 13.646, 13.846, ..., 10.346, 10.646, 11.046],
        [12.446, 12.946, 13.946, ...,  9.246,  9.946, 10.546],
        [12.546, 12.946, 13.346, ...,  8.946,  9.246,  9.646],
        ...,
        [ 5.746,  5.246,  4.546, ..., 11.746,  8.846,  4.846],
        [ 5.846,  5.546,  4.746, ...,  5.746,  1.846, -0.354],
        [ 6.146,  5.746,  5.046, ..., -2.354, -4.054, -4.754]],

       [[13.35 , 13.77 , 14.06 , ...,  9.38 ,  9.55 ,  9.63 ],
        [12.78 , 13.91 , 14.52 , ...,  8.33 ,  8.13 ,  8.47 ],
        [12.82 , 13.7  , 14.77 , ...,  6.69 ,  7.26 ,  8.14 ],
        ...,
        [ 6.

In [58]:
lon = [attrs['x_llcenter'] + attrs['dx'] * i for i in np.arange(attrs['n_cols'])]
lon

[160.0,
 160.5,
 161.0,
 161.5,
 162.0,
 162.5,
 163.0,
 163.5,
 164.0,
 164.5,
 165.0,
 165.5,
 166.0,
 166.5,
 167.0,
 167.5,
 168.0,
 168.5,
 169.0,
 169.5,
 170.0,
 170.5,
 171.0,
 171.5,
 172.0,
 172.5,
 173.0,
 173.5,
 174.0,
 174.5,
 175.0,
 175.5,
 176.0,
 176.5,
 177.0,
 177.5,
 178.0,
 178.5,
 179.0]

In [59]:
lat = [attrs['y_llcenter'] + attrs['dy'] * i for i in np.arange(attrs['n_rows'])]
lat

[-49.0,
 -48.5,
 -48.0,
 -47.5,
 -47.0,
 -46.5,
 -46.0,
 -45.5,
 -45.0,
 -44.5,
 -44.0,
 -43.5,
 -43.0,
 -42.5,
 -42.0,
 -41.5,
 -41.0,
 -40.5,
 -40.0,
 -39.5,
 -39.0,
 -38.5,
 -38.0,
 -37.5,
 -37.0,
 -36.5,
 -36.0,
 -35.5,
 -35.0,
 -34.5,
 -34.0,
 -33.5,
 -33.0]

In [60]:
da = xr.DataArray(data, dims=['time','y','x'],
                     coords={'time': d2.time, 'y':lat, 'x':lon}, name='msl')

In [61]:
da

<xarray.DataArray 'msl' (time: 17, y: 33, x: 39)>
array([[[12.573, 13.373, ..., 12.373, 12.673],
        [11.873, 12.573, ..., 11.473, 11.873],
        ...,
        [ 4.273,  4.073, ..., -5.427, -5.727],
        [ 4.473,  4.273, ..., -5.627, -5.327]],

       [[13.346, 13.646, ..., 10.646, 11.046],
        [12.446, 12.946, ...,  9.946, 10.546],
        ...,
        [ 5.846,  5.546, ...,  1.846, -0.354],
        [ 6.146,  5.746, ..., -4.054, -4.754]],

       ...,

       [[-0.885, -0.675, ...,  6.555,  5.375],
        [-0.255, -0.305, ...,  6.195,  5.275],
        ...,
        [ 8.725,  8.705, ..., -0.985, -1.215],
        [ 9.325,  9.345, ..., -0.425, -0.665]],

       [[-1.201, -1.931, ...,  8.349,  7.419],
        [-0.781, -1.431, ...,  7.199,  7.359],
        ...,
        [ 9.289,  9.209, ..., -0.771, -0.751],
        [ 9.939,  9.659, ..., -0.101, -0.241]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-09-01 2018-09-01T03:00:00 ... 2018-09-03
  * y        (y) float64 -49.0 